# Try `osm`, `osmnx` packages

## Check if Open Street Map has good CA coverage (`osm`)

Would be nice to use `maxspeed` parameter.

Let's see where the waypoints and nodes are?

* https://pyrosm.readthedocs.io/en/latest/installation.html

In [ ]:
#!pip install osmium pyrosm ogr

In [ ]:
from pyrosm.data import sources

# Print available source categories
sources.available.keys()

In [ ]:
from pyrosm import get_data, OSM

# Pyrosm comes with a couple of test datasets 
# that can be used straight away without
# downloading anything
fp = get_data("seattle")

# Initialize the OSM parser object
osm = OSM(fp)

# Read all drivable roads
# =======================
#drive_net = osm.get_network(network_type="driving")
#drive_net.plot()

In [ ]:
print(sources.subregions.usa.available)

In [ ]:
from pyrosm import OSM
osm_path ='../california-latest.osm.pbf' # downloaded from geofabrik
osm = OSM(osm_path)

In [ ]:
drive_net = osm.get_network(network_type="driving")
drive_net.plot()

In [ ]:
import osmium as osm
import pandas as pd

In [ ]:
#https://stackoverflow.com/questions/45771809/how-to-extract-and-visualize-data-from-osm-file-in-python
#http://andrewgaidus.com/Convert_OSM_Data/
#https://github.com/agaidus/Converting_OSM_Data/blob/master/Convert_OSM_Data.ipynb
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type):
        for tag in elem.tags:
            self.osm_data.append([elem_type, 
                                   elem.id, 
                                   elem.version,
                                   elem.visible,
                                   pd.Timestamp(elem.timestamp),
                                   elem.uid,
                                   elem.user,
                                   elem.changeset,
                                   len(elem.tags),
                                   tag.k, 
                                   tag.v])

    def node(self, n):
        self.tag_inventory(n, "node")

    def way(self, w):
        self.tag_inventory(w, "way")

    def relation(self, r):
        self.tag_inventory(r, "relation")

In [ ]:
osmhandler = OSMHandler()

#FILE = "../california-latest.osm.pbf"
#FILE = "/tmp/pyrosm/socal-latest.osm.pbf"
FILE = "/tmp/pyrosm/Seattle.osm.pbf"

# scan the input file and fills the handler list accordingly
osmhandler.apply_file(FILE)

In [ ]:
#https://techoverflow.net/2021/04/25/minimal-example-how-to-read-osm-pbf-file-using-python-osmium/
print(f'Number of nodes: {osmhandler.node_count}')
print(f'Number of way: {osmhandler.way_count}')
print(f'Number of relations: {osmhandler.relation_count}')

In [ ]:
class StreamHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.edge_and_nodes = [] 
    def way(self, w): 
        yield {'edge_id': w.id, 
               'nodes': [w.nodes[i].ref for i in range(len(w.nodes))]}
h = StreamHandler()
h.apply_file(FILE)
#for row in h.way(w): 
#    print(row) 

for i in h.way(w):
    print(i)

In [ ]:
# transform the list into a pandas DataFrame
data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
                 'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
#df_osm = tag_genome.sort_values(by=['type', 'id', 'ts'])

In [ ]:
df_osm.head()

In [ ]:
# In terminal:
# wget -c http://download.geofabrik.de/north-america/us/california-latest.osm.pbf

# Tutorial: https://www.linuxbabe.com/linux-server/openstreetmap-tile-server-ubuntu-16-04
# https://stackoverflow.com/questions/45771809/how-to-extract-and-visualize-data-from-osm-file-in-python

## `osmnx`

Issue here is that bus stops get attached to the same nearest node, even every 5th bus stop. 

In [ ]:
import networkx as nx
import osmnx as ox
import pandas as pd

In [ ]:
G = ox.graph_from_place("Piedmont, California, USA", network_type="drive")
fig, ax = ox.plot_graph(G)

In [ ]:
def nearest_node(row):
    node = ox.get_nearest_node(G, (row.geometry.y, row.geometry.x))
    return node

In [ ]:
df["node"] = df.apply(lambda x: nearest_node(x), axis=1) 

In [ ]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
gdf_nodes.head()

In [ ]:
# get the nearest network node to each point
orig_node = ox.get_nearest_node(G, (37.828903, -122.245846))
dest_node = ox.get_nearest_node(G, (37.812303, -122.215006))

# how long is our route in meters?
#nx.shortest_path_length(G, orig_node, dest_node, weight='length')

In [ ]:
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
#(G, orig, dest, weight="travel_time")


In [ ]:
#https://github.com/gboeing/osmnx-examples/blob/main/notebooks/02-routing-speed-time.ipynb
hwy_speeds = {
    "residential": 35, #21 mph
    "secondary": 50, #31 mph
    "tertiary": 60, #37 mph
}
# What should speeds be set to for buses?
# For now, hard-code, but probably can get some max speeds from Eric's RT
# to ground-truth actual max speeds for buses
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

In [ ]:
edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)